In [1]:
import pandas as pd

TEST = '/kaggle/input/playground-series-s4e4/test.csv'
TRAIN = '/kaggle/input/playground-series-s4e4/train.csv'
test_df = pd.read_csv(filepath_or_buffer=TEST, index_col=['id']).drop(columns=['Sex'])

train_df = pd.read_csv(filepath_or_buffer=TRAIN, index_col=['id']).drop(columns=['Sex'])
# do we want to drop outliers from the training data ?
# train_df = train_df[train_df['Height'] < 0.24]

train_df.head()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [2]:
import warnings
from plotly import express

warnings.filterwarnings(action='ignore', category=FutureWarning)
if 'Sex' in train_df.columns:
    express.histogram(data_frame=train_df, x='Rings', facet_col='Sex')

In [3]:
import arrow
from sklearn.linear_model import LogisticRegression

COLUMNS = ['Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1', 'Whole weight.2', 'Shell weight',]
TARGET = 'Rings'
TOL = 1e-12

time_start = arrow.now()

logreg = LogisticRegression(penalty='l2', dual=False, tol=TOL, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None,
                           random_state=2024, solver='lbfgs', max_iter=10000, multi_class='auto', verbose=1, warm_start=False, n_jobs=None,
                           l1_ratio=None).fit(X=train_df[COLUMNS], y=train_df[TARGET])
print('{} model fit complete in {} iterations'.format(arrow.now() - time_start, logreg.n_iter_))
logreg_df = pd.DataFrame(data={'id' : test_df.index.tolist(), 'Rings': logreg.predict(X=test_df[COLUMNS])})
print('{} model done'.format(arrow.now() - time_start))

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          224     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.01948D+05    |proj g|=  1.42288D+04


 This problem is unconstrained.



At iterate   50    f=  1.68723D+05    |proj g|=  5.47756D+02

At iterate  100    f=  1.64920D+05    |proj g|=  2.06417D+02

At iterate  150    f=  1.63888D+05    |proj g|=  1.65748D+02

At iterate  200    f=  1.63687D+05    |proj g|=  5.92529D+01

At iterate  250    f=  1.63609D+05    |proj g|=  2.80957D+01

At iterate  300    f=  1.63584D+05    |proj g|=  1.42689D+01

At iterate  350    f=  1.63575D+05    |proj g|=  3.05080D+01

At iterate  400    f=  1.63573D+05    |proj g|=  3.97145D+00

At iterate  450    f=  1.63572D+05    |proj g|=  2.80380D+00

At iterate  500    f=  1.63571D+05    |proj g|=  2.02689D+00

At iterate  550    f=  1.63571D+05    |proj g|=  1.92321D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F   

In [4]:
express.histogram(x=COLUMNS, y=logreg.coef_[0])

Let's make a split and start comparing models.

In [5]:
import arrow
import math
from sklearn.linear_model import LogisticRegression
# we don't have the RMSLE directly, so we have to use the MSLE
# and post-process to get the RMSLE
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

time_start = arrow.now()
X_train, X_test, y_train, y_test = train_test_split(train_df[COLUMNS], train_df[TARGET], test_size=0.2, random_state=2024, stratify=train_df[TARGET])

regression = LogisticRegression(max_iter=1000, tol=1e-12).fit(X=X_train, y=y_train)
print('fit complete after {} iterations.'.format(regression.n_iter_[0]))
print('RMSLE: {:6.5f}'.format(math.sqrt(mean_squared_log_error(y_true=y_test, y_pred=regression.predict(X=X_test)))))
print('done in {}'.format(arrow.now() - time_start))

fit complete after 542 iterations.
RMSLE: 0.18384
done in 0:00:51.807035


Let's build and compare a bunch of regressors.

In [6]:
import arrow
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor

MODELS = {
    'Logistic Regression': LogisticRegression(max_iter=1000, tol=1e-12),   
}

for n_neighbors in range(70, 90): 
    MODELS['KNN k = {}'.format(n_neighbors)] = KNeighborsRegressor(n_neighbors=n_neighbors, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)

scores = []
for name, regressor in MODELS.items():
    time_start = arrow.now()
    regressor.fit(X=X_train, y=y_train)
    score = math.sqrt(mean_squared_log_error(y_true=y_test, y_pred=regressor.predict(X=X_test)))
    print('{:6.5f} : {} {}'.format(score, name, arrow.now() - time_start))
    scores.append((score, name))

print(sorted(scores)[0])

0.18384 : Logistic Regression 0:00:52.312680
0.15313 : KNN k = 70 0:00:01.849516
0.15311 : KNN k = 71 0:00:01.772011
0.15310 : KNN k = 72 0:00:01.687755
0.15311 : KNN k = 73 0:00:01.770219
0.15311 : KNN k = 74 0:00:01.805749
0.15310 : KNN k = 75 0:00:01.728183
0.15310 : KNN k = 76 0:00:01.766185
0.15312 : KNN k = 77 0:00:01.757596
0.15309 : KNN k = 78 0:00:01.789432
0.15308 : KNN k = 79 0:00:01.787435
0.15308 : KNN k = 80 0:00:01.807982
0.15311 : KNN k = 81 0:00:01.818589
0.15311 : KNN k = 82 0:00:01.840262
0.15310 : KNN k = 83 0:00:01.820621
0.15310 : KNN k = 84 0:00:01.814848
0.15309 : KNN k = 85 0:00:01.841391
0.15310 : KNN k = 86 0:00:01.833577
0.15312 : KNN k = 87 0:00:01.873344
0.15312 : KNN k = 88 0:00:01.862073
0.15313 : KNN k = 89 0:00:01.843548
(0.15308106850508654, 'KNN k = 79')


In [7]:
import arrow
from sklearn.neighbors import KNeighborsRegressor

time_start = arrow.now()
knn = KNeighborsRegressor(n_neighbors=79, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
knn.fit(X=train_df[COLUMNS], y=train_df[TARGET])
knn_df = pd.DataFrame(data={'id' : test_df.index.tolist(), 'Rings': knn.predict(X=test_df[COLUMNS])})
print('{} model done'.format(arrow.now() - time_start))

0:00:06.810883 model done


In [8]:
knn_df.to_csv(path_or_buf='/kaggle/working/submission.csv', index=False)